In [ ]:
import numpy as np
from scipy.special import factorial
from scipy.misc import derivative
import matplotlib.pyplot as plt

# --- 1. Define Model Hyperparameters ---
# These can be changed to explore different scenarios.
params = {
    "d": 20,          # Input dimension
    "k_parity": 2,    # Sparsity of the target parity function (k)
    "N": 1000,         # Number of neurons in the hidden layer
    "gamma": 0.0,     # Scaling exponent for the output weights' prior
    "kappa": 0.05,     # Noise parameter in the loss function
    "sigma_v": 1.0,   # Variance parameter for input weights w
    "sigma_w": 1.0,   # Variance parameter for output weights a
}


# --- 2. Special Handling for ReLU Taylor Coefficients ---
# The pure ReLU function is not smooth at z=0, so its Taylor series
# is ill-defined. In theoretical models, one typically considers a
# smoothed version. Here, we use numerical differentiation on a
# slightly smoothed ReLU to find the coefficients c_k = phi^(k)(0) / k!

def smoothed_relu(z, delta=1e-4):
    """A smooth approximation of the ReLU function."""
    return (z + np.sqrt(z**2 + delta**2)) / 2

def get_relu_taylor_coeff(k):
    """
    Numerically calculates the k-th Taylor coefficient of the smoothed ReLU.
    c_k = phi^(k)(0) / k!
    """
    if k == 0:
        return 0.0 # ReLU(0) = 0
    
    # Use numerical differentiation to find the k-th derivative at z=0
    # dx=1e-2 is a good step size for numerical stability here.
    k_th_derivative_at_0 = derivative(smoothed_relu, 0.0, n=k, dx=1e-2, order=k+1+(k%2))
    
    return k_th_derivative_at_0 / factorial(k)

# --- 3. Method 1: Analytical Calculation of P_c ---

def calculate_Pc_analytical(params):
    """
    Calculates P_c using the analytical formulas for the average quantities.
    """
    d = params["d"]
    k = params["k_parity"]
    N = params["N"]
    gamma = params["gamma"]
    kappa = params["kappa"]
    sigma_v = params["sigma_v"]
    sigma_w = params["sigma_w"]

    # Get the necessary Taylor coefficient for the target parity k
    c_k = get_relu_taylor_coeff(k)
    if np.abs(c_k) < 1e-9:
        print(f"Warning: Taylor coefficient c_{k} is nearly zero. "
              "The model may not be able to learn this parity. P_c will be infinite.")
        return np.inf
        
    # a) Calculate average self-energy <Sigma>
    # For ReLU, the constant alpha_phi is 1/2.
    alpha_phi = 0.5 
    avg_Sigma = alpha_phi * sigma_v**2
    
    # b) Calculate average squared projection onto the target <J_S^2>
    avg_J_S_sq = (c_k * factorial(k))**2 * (sigma_v**2 / d)**k
    
    # c) Calculate average interference noise <Upsilon> using the approximation
    avg_Upsilon = avg_Sigma - avg_J_S_sq
    
    # d) Calculate P_c using the final derived formula
    # P_c = <Υ> / [ (κ*N^γ/σ_w²) + <Σ> - N*<J_S²> ]
    
    # Note: This is one of the possible final formulas. Different conventions in the
    # base action can change the powers of kappa. We use a self-consistent one here.
    denominator = (kappa * N**gamma / sigma_w**2) + avg_Sigma - N * avg_J_S_sq
    
    if denominator <= 0:
        print("Learning is always favorable or at a critical point (denominator <= 0). P_c is effectively 0.")
        return 0
        
    P_c = avg_Upsilon / denominator
    
    return P_c


# --- 4. Method 2: Monte Carlo Estimation of P_c ---

def calculate_Pc_monte_carlo(params, num_w_samples=10000, num_x_samples=500):
    """
    Estimates P_c by sampling random weight vectors `w` and inputs `x`.
    """
    d = params["d"]
    k = params["k_parity"]
    N = params["N"]
    gamma = params["gamma"]
    kappa = params["kappa"]
    sigma_v = params["sigma_v"]
    sigma_w = params["sigma_w"]

    # Accumulators for our averages
    acc_Sigma = 0.0
    acc_J_S_sq = 0.0
    
    # Generate a fixed sparse set S for the target parity
    S_indices = np.random.choice(d, k, replace=False)

    print("Running Monte Carlo simulation... (this may take a moment)")
    for _ in range(num_w_samples):
        # Sample a random weight vector w
        w = np.random.randn(d) * (sigma_v / np.sqrt(d))
        
        # Estimate Sigma(w) = 0.5 * ||w||^2
        Sigma_w = 0.5 * np.sum(w**2)
        acc_Sigma += Sigma_w
        
        # Estimate J_S(w) = E_x[ReLU(w^T*x) * chi_S(x)] via sampling x
        # This is a nested Monte Carlo step
        acc_J_S_w = 0.0
        for _ in range(num_x_samples):
            x = np.random.choice([-1, 1], size=d)
            z = np.dot(w, x)
            relu_z = np.maximum(0, z)
            chi_S = np.prod(x[S_indices])
            acc_J_S_w += relu_z * chi_S
        
        J_S_w = acc_J_S_w / num_x_samples
        acc_J_S_sq += J_S_w**2
        
    # Finalize the averages
    avg_Sigma_mc = acc_Sigma / num_w_samples
    avg_J_S_sq_mc = acc_J_S_sq / num_w_samples
    
    # Calculate Upsilon from the estimated averages
    avg_Upsilon_mc = avg_Sigma_mc - avg_J_S_sq_mc
    
    # Calculate P_c using the same final formula as the analytical method
    denominator = (kappa * N**gamma / sigma_w**2) + avg_Sigma_mc - N * avg_J_S_sq_mc
    
    if denominator <= 0:
        print("MC Estimation: Learning is always favorable (denominator <= 0). P_c is effectively 0.")
        return 0
        
    P_c_mc = avg_Upsilon_mc / denominator
    
    return P_c_mc


# --- 5. Main Execution ---

if __name__ == '__main__':
    print("--- Calculating Critical Dataset Size P_c ---\n")
    print(f"Hyperparameters: {params}\n")

    # Analytical result
    print("Method 1: Using Analytical Approximations")
    P_c_analytic = calculate_Pc_analytical(params)
    print(f"  --> Analytical P_c = {P_c_analytic:.2f}\n")

    # Monte Carlo result
    print("Method 2: Using Monte Carlo Estimation")
    P_c_mc = calculate_Pc_monte_carlo(params)
    print(f"  --> Monte Carlo P_c = {P_c_mc:.2f}\n")
    
    print("Note: The two results should be close. Differences arise from MC sampling error and approximations in the analytical formulas.")